# Toronto Neighborhoods Excercise - Capstone Week 3
### Import Libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

In [2]:
from IPython.display import display_html
import pandas as pd
import numpy as np
import json 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library
from bs4 import BeautifulSoup as bs
import urllib.request
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 9.5MB/s eta 0:00:01
Folium installed
Libraries imported.


### Pull all tables from wiki site into data frame

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',flavor='html5lib')
for df in dfs:
    print(df.head())  

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
                                                  0   \
0                                                NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NL   
3                                                  A   

                                                  1   \
0                              Canadian postal codes   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NS   
3                                                  B   

                                                  2    3    4    5    6    7   \
0  

### Generate desired data from available dataframe options

In [4]:
df=dfs[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Remove "Not Assigned" Boroughs

In [5]:
df_clean = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_clean.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Group Neighborhoods with same borough

In [6]:
df_grouped = df_clean.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assign "Not Assigned" Neighborhoods to Borough Name

In [7]:
for index, row in df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Shape It Up!!

In [8]:
df_grouped.shape

(103, 3)

### Import Lat/Long CSV Data

In [9]:
lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Lat/Long Data with Wiki Data

In [10]:
df_merged = pd.merge(df_grouped,lat_long, on='Postal Code')
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Generate Neighborhood Map

In [11]:
#Initial Neighborhood Map
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df_merged['Latitude'],df_merged['Longitude'],df_merged['Borough'],df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [12]:
# The code was removed by Watson Studio for sharing.

## Data Exploration
### Define Function to Gather Venue Data from Foursquare

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Call function to gather venue data for Toronto

In [14]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'], 
                                 latitudes=df_merged['Latitude'], 
                                 longitudes=df_merged['Longitude']
                                )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

### Check shape and header info of new data frame

In [15]:
#new df parameters
print(toronto_venues.shape)
toronto_venues.head()

(138, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
1,Scarborough Village,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
2,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant
3,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Price Chopper,43.799445,-79.318563,Grocery Store
4,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Subway,43.798671,-79.318475,Sandwich Place


### Group venues by Neighborhood

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",3,3,3,3,3,3
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Canada Post Gateway Processing Centre,1,1,1,1,1,1
Central Bay Street,4,4,4,4,4,4
"Commerce Court, Victoria Hotel",16,16,16,16,16,16
Davisville,8,8,8,8,8,8
Don Mills,2,2,2,2,2,2
Downsview,1,1,1,1,1,1


### Count Unique Venues

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 76 uniques categories.


### One Hot Coding

In [18]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Steeles West, L'Amoreaux West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Steeles West, L'Amoreaux West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Steeles West, L'Amoreaux West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by Neighborhood and Mean Freq of Venue

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
1,"Bedford Park, Lawrence Manor East",0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
3,Canada Post Gateway Processing Centre,0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
4,Central Bay Street,0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.00,0.0625,0.0625,0.000000,0.0625,0.0,0.000000,0.000000,0.0625,...,0.062500,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
6,Davisville,0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.125000,0.000000,0.000000,0.000000,0.0,0.0,0.125,0.0,0.000000,0.000000
7,Don Mills,0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
8,Downsview,0.00,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000,0.000000
9,"Fairview, Henry Farm, Oriole",0.00,0.0000,0.0000,0.000000,0.0000,0.5,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.000,0.0,0.000000,0.000000


### Define function to sort data

In [20]:
#Function to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


### Get the top 10 Venues

In [21]:
#Top 10 Venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Fried Chicken Joint,Deli / Bodega,Yoga Studio,Dessert Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
1,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Yoga Studio,Deli / Bodega,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
2,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Yoga Studio,Deli / Bodega,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Dessert Shop
3,Canada Post Gateway Processing Centre,Coffee Shop,Yoga Studio,Dessert Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
4,Central Bay Street,Coffee Shop,Pharmacy,Sandwich Place,Yoga Studio,Clothing Store,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop


## K-Means Analysis

In [22]:
## K-Means Analysis ##
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 4, 4, 2, 2, 2, 0, 2], dtype=int32)

In [23]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Golf Course,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,43.770992,-79.216917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
toronto_merged = toronto_merged.dropna()
toronto_merged = toronto_merged.astype({'Cluster Labels':'int32'})
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Golf Course,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Deli / Bodega,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Yoga Studio
15,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.799525,-79.318389,2,Grocery Store,Chinese Restaurant,Pharmacy,Coffee Shop,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Cupcake Shop,College Gym,Concert Hall
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,Theater,Bank,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
26,M3B,North York,Don Mills,43.745906,-79.352188,2,Pool,Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop


### Map of Clusters

In [40]:
## Map Our New Clusters ###
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Cluster 1

In [41]:
### Cluster 1 ###
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,North York,0,Construction & Landscaping,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
31,North York,0,Construction & Landscaping,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
32,North York,0,Construction & Landscaping,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
33,North York,0,Construction & Landscaping,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner


### Examine Cluster 2

In [42]:
### Cluster 2 ###
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Intersection,Park,Yoga Studio,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop
50,Downtown Toronto,1,Park,Yoga Studio,Deli / Bodega,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Dessert Shop


### Examine Cluster 3

In [43]:
### Cluster 3 ###
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Golf Course,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
5,Scarborough,2,Playground,Deli / Bodega,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Yoga Studio
15,Scarborough,2,Grocery Store,Chinese Restaurant,Pharmacy,Coffee Shop,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Cupcake Shop,College Gym,Concert Hall
18,North York,2,Theater,Bank,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
26,North York,2,Pool,Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
27,North York,2,Pool,Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
37,East Toronto,2,Trail,Yoga Studio,Dessert Shop,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
39,East York,2,Indian Restaurant,Housing Development,Sandwich Place,Intersection,Coffee Shop,Fast Food Restaurant,Farmers Market,Diner,Dessert Shop,Food Court
47,Central Toronto,2,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Seafood Restaurant,Café,Toy / Game Store,Fast Food Restaurant,Farmers Market,Diner
49,Central Toronto,2,Supermarket,Liquor Store,Clothing Store,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega


### Examine Cluster 4

In [44]:
### Cluster 4 ###
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,3,Italian Restaurant,Yoga Studio,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop
72,North York,3,Italian Restaurant,Yoga Studio,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop


### Examine Cluster 5

In [45]:
### Cluster 5 ###
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,4,Coffee Shop,Fried Chicken Joint,Deli / Bodega,Yoga Studio,Dessert Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
54,Downtown Toronto,4,Coffee Shop,Art Gallery,Diner,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop
57,Downtown Toronto,4,Coffee Shop,Pharmacy,Sandwich Place,Yoga Studio,Clothing Store,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
59,Downtown Toronto,4,Coffee Shop,Gym,Yoga Studio,Dessert Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
60,Downtown Toronto,4,Coffee Shop,Restaurant,Deli / Bodega,Gym,Yoga Studio,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
62,North York,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Yoga Studio,Deli / Bodega,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
86,Mississauga,4,Coffee Shop,Yoga Studio,Dessert Shop,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner
99,Etobicoke,4,Pizza Place,Coffee Shop,Yoga Studio,Deli / Bodega,College Gym,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Dessert Shop


## Cluster Definition from Analysis

* Cluster 1: North York

* Cluster 2: Business District

* Cluster 3: Urban Living

* Cluster 4: Little Italy

* Cluster 5: Hipsters